# 5.1 Evaluating generative text models

## 5.1.1 Using GPT to generate text

In [1]:
# Import code from last chapter and initialize a GPT model
import torch
from previous_code_5 import GPTModel
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256, # Changed from 1024 to 256 to reduce computational demands
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

In [2]:
# Utility functions for text to token ID conversion
import tiktoken
from previous_code_5 import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special = {'<|endoftext|>'})
    return torch.tensor(encoded).unsqueeze(0)

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

In [3]:
# Test the two above functions
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(model = model, idx = text_to_token_ids(start_context, tokenizer), 
                                 max_new_tokens = 10, context_size = GPT_CONFIG_124M["context_length"])
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingFactor incarnationDimCreated beachesPolitics wage Constant conco


## 5.1.2 Calculating the text generation loss

In [9]:
# Input examples
inputs = torch.tensor([[16833, 3626, 6100], # every efffort moves
                       [40, 1107, 588]]) # I really like
targets = torch.tensor([[3626, 6100, 345], # effort moves you
                        [1107, 588, 11311]]) # really like chocolate

In [10]:
# Get probas
with torch.no_grad():
    logits = model(inputs)
probas = torch.softmax(logits, dim = -1)
print(probas.shape)

torch.Size([2, 3, 50257])


In [11]:
# Obtain corresponding token IDs
token_ids = torch.argmax(probas, dim = -1, keepdim = True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16657],
         [45647],
         [ 1804]],

        [[49906],
         [15335],
         [20861]]])


In [12]:
# Converts the token IDs back into text
print(f"Targes batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Output batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targes batch 1:  effort moves you
Output batch 1:  Armed sd doing


In [14]:
# Print the initial softmax probability scores
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([9.3955e-05, 2.8011e-05, 1.8640e-05])
Text 2: tensor([1.1259e-05, 4.3579e-05, 5.9627e-06])


In [15]:
# Applying the logarithm to the probability scores
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.2727, -10.4829, -10.8902, -11.3943, -10.0409, -12.0300])


In [16]:
# Combine the log probabilities into a single score by averaging
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor(-10.6852)


In [17]:
# Calculate the negative average log probability
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.6852)


In [18]:
# Get the shape of the logits and target tensors
print("Logits shape:", logits.shape)
print("Targets shape:", targets.shape)

Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])


In [19]:
# Flatten the tensors by combining them over the batch dimension
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()
print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


In [20]:
# Calculate loss with cross_entropy function
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.6852)


In [21]:
# Calculate the perplexity
perplexity = torch.exp(loss)
print(perplexity)

tensor(43702.9180)
